# Load configuration

In [1]:
import os
import yaml


with open("../conf/service.dev.yaml", 'r') as f:
    configs = yaml.safe_load(f)
os.environ['OPENAI_API_KEY'] = configs['openai']['api_key']

# Chain = Prompt | Model | Parser

## 1. Basic chain
1. Prompt
    - `from_template`
2. Model
    - `ChatOpenAI`
3. Parser
    - `StrOutputParser`

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

chain.invoke({'topic': "bears"})

"Why don't bears wear shoes?\n\nBecause they already have bear feet!"

## 2. Function calling
1. Prompt
    - `from_messages`
2. Model
    - `ChatOpenAI`
    - `functions`
3. Parser
    - `OpenAIFunctionsAgentOutputParser`

### 2.1 Define input class(`pydantic`)

In [3]:
from pydantic import BaseModel, Field


class WikipediaInput(BaseModel):
    query: str = Field(description="Query to search")
    n_contents: int = Field(description="Number of contents to return")

### 2.2 Define python function(`tool`)

In [4]:
from langchain.agents import tool
import wikipedia
from wikipedia.exceptions import PageError, DisambiguationError


@tool(args_schema=WikipediaInput)
def search_wikipedia(query: str, n_contents: int) -> str:
    """Run Wikipedia search and get page summaries."""
    titles = wikipedia.search(query)
    summaries = []

    for title in titles[:n_contents]:
        try:
            page = wikipedia.page(title=title, auto_suggest=False)
            summary = page.summary
            summaries.append(f"[{title}]"
                             f"{summary}")
        except (PageError, DisambiguationError):
            pass
    
    if not summaries:
        return "No good Wikipedia Search Result was found"
    
    return (50*"=").join(summaries)

### 2.3 Convert to OpenAI function(schema)

In [5]:
from langchain.tools.render import format_tool_to_openai_function

tools = [search_wikipedia]
functions = [format_tool_to_openai_function(tool) for tool in tools]
functions

/home/vscode/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


[{'name': 'search_wikipedia',
  'description': 'search_wikipedia(query: str, n_contents: int) -> str - Run Wikipedia search and get page summaries.',
  'parameters': {'type': 'object',
   'properties': {'query': {'description': 'Query to search',
     'type': 'string'},
    'n_contents': {'description': 'Number of contents to return',
     'type': 'integer'}},
   'required': ['query', 'n_contents']}}]

### 2.4 Build chain

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


prompt = ChatPromptTemplate.from_messages([
    ('system', "You are helpful but sassy assistant"),
    ('user', "{input}")
])
model = ChatOpenAI()
model = model.bind(functions=functions)
parser = OpenAIFunctionsAgentOutputParser()
chain = prompt | model | parser

print(chain.invoke({'input': "Hi!"}))
print(chain.invoke({'input': "what is langchain?"}))

return_values={'output': 'Hello! How can I assist you today?'} log='Hello! How can I assist you today?'
tool='search_wikipedia' tool_input={'query': 'langchain', 'n_contents': 1} log="\nInvoking: `search_wikipedia` with `{'query': 'langchain', 'n_contents': 1}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "langchain",\n  "n_contents": 1\n}', 'name': 'search_wikipedia'}})]


# 3. Fallback handling

1. `json.loads` 가 정확한  json format을 입력받지 못해 에러가 발생

In [7]:
from langchain.llms import OpenAI
import json


main_model = OpenAI(
    temperature=0,
    max_tokens=1000,
    model='gpt-3.5-turbo-instruct'
)
main_chain = main_model | json.loads

challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"
main_chain.invoke(challenge)

/home/vscode/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


JSONDecodeError: Extra data: line 9 column 1 (char 125)

2. 최신 LLM 사용방식은 정확한 json format을 줄 수 있음
    1. `langchain.llms.OpenAI`의 output은 `str`이고, \
    `langchain_openai.ChatOpenAI`의 output은 `AIMessage` 객채
    2. Output의 format을 `str`로 맞춰주기 위해 `StrOutputParser` 추가

In [8]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
import json


fallback_model = ChatOpenAI(temperature=0)
fallback_chain = fallback_model | StrOutputParser() | json.loads
fallback_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'In the stillness of night, whispers of the wind'},
 'poem2': {'title': 'Silent Tears',
  'author': 'Jacob Anderson',
  'first_line': 'Silent tears fall, unseen and unheard'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Lee',
  'first_line': 'Beneath the moonlight, dancing shadows sway'}}

3. 정상작동하는 2번 방법을 fallback method로 지정

In [9]:
final_chain = main_chain.with_fallbacks([fallback_chain])
final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

# 4. Batch processing
가능한 batch 처리되는 프로세스들은 병렬적으로 실행된다. (default `n_procs=5`)

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

chain.batch([
    {'topic': "bears"},
    {'topic': "frogs"}
])

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 'Why don\'t frogs make good loan officers?\n\nBecause they always end up in "debt"!']

# 5. Streaming processing
적은 지연시간은 더 나은 사용자 경험을 만들어낼 수 있다.

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

for t in chain.stream({'topic': "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 already
 have
 bear
 feet
!



# 6. Asynchronous methoes
용도에 따라 비동기적 처리도 가능 (`ainvoke`, `abatch`, `astream`)

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

response = await chain.ainvoke({'topic': "bears"})
response

"Why don't bears wear shoes?\n\nBecause they have bear feet!"